In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("/home/maitanha/cll_vlm/cll_vlm/ol_cll_logs/stage2/cifar20_correspond_stage2.csv")


In [4]:
df.head()

,index,true_label,random_label,predicted,raw_answer,clip_pos_similarity,clip_neg_similarity
0,0,large_omnivores_and_herbivores,people,CL,NO,0.1733,0.1770
1,1,reptiles,non-insect_invertebrates,OL,YES,0.1830,0.1853
2,2,fruit_and_vegetables,people,CL,NO,0.1847,0.1917
3,3,people,trees,CL,NO,0.1481,0.1753
4,4,fish,household_electrical_devices,CL,NO,0.1289,0.1686


In [6]:
df["pos > neg"] = df["clip_pos_similarity"] > df["clip_neg_similarity"]
df["neg > pos"] = df["clip_neg_similarity"] > df["clip_pos_similarity"]
df["pos == neg"] = df["clip_pos_similarity"] == df["clip_neg_similarity"]

# Phân tích cho : predicted = OL

In [28]:
ol_df = df[df["predicted"] == "OL"]

ol_total = len(ol_df)

ol_correct = df[
    (df["predicted"] == "OL") &
    (df["true_label"] == df["random_label"])
]

ol_pos_win = ol_df["pos > neg"].sum()
ol_neg_win = ol_df["neg > pos"].sum()
ol_equal   = ol_df["pos == neg"].sum()

print(f"OL total: {ol_total}")
print(f"OL correct: {len(ol_correct)} ({len(ol_correct)/ol_total:.2%})")
print(f"OL pos > neg: {ol_pos_win} ({ol_pos_win/ol_total:.2%})")
print(f"OL neg > pos: {ol_neg_win} ({ol_neg_win/ol_total:.2%})")
print(f"OL equal:     {ol_equal} ({ol_equal/ol_total:.2%})")


print("\n==================================================\n")

ol_correct = df[
    (df["predicted"] == "OL") &
    (df["true_label"] == df["random_label"])
]

ol_correct_total = len(ol_correct)
ol_correct_pos_win = ol_correct["pos > neg"].sum()

print(f"OL correct & pos > neg : {ol_correct_pos_win}/{ol_correct_total} "
      f"({ol_correct_pos_win / ol_correct_total:.2%})")


print("\n==================================================\n")

ol_pos_win_df = df[
    (df["predicted"] == "OL") &
    (df["pos > neg"])
]

ol_pos_win_total = len(ol_pos_win_df)

ol_pos_win_correct = (
    ol_pos_win_df["true_label"] == ol_pos_win_df["random_label"]
).sum()

print(f"[predicted = OL] & [pos > neg] & [true_label = random_label]: {ol_pos_win_correct}/{ol_pos_win_total} "
      f"({ol_pos_win_correct/ol_pos_win_total:.2%})")

print("\n==================================================\n")

ol_neg_win_df = df[
    (df["predicted"] == "OL") &
    (df["neg > pos"])
]

ol_neg_win_total = len(ol_neg_win_df)

ol_neg_win_wrong = (
    ol_neg_win_df["true_label"] != ol_neg_win_df["random_label"]
).sum()

print(f"[predicted = OL] & [neg > pos] & [true_label != random_label]: {ol_neg_win_wrong}/{ol_neg_win_total} "
      f"({ol_neg_win_wrong/ol_neg_win_total:.2%})")

OL total: 6615
OL correct: 1830 (27.66%)
OL pos > neg: 3968 (59.98%)
OL neg > pos: 2633 (39.80%)
OL equal:     14 (0.21%)


OL correct & pos > neg : 1491/1830 (81.48%)


[predicted = OL] & [pos > neg] & [true_label = random_label]: 1491/3968 (37.58%)


[predicted = OL] & [neg > pos] & [true_label != random_label]: 2295/2633 (87.16%)


In [35]:
df["clip_margin"] = df["clip_pos_similarity"] - df["clip_neg_similarity"]

In [36]:
ol_pos_win_df = df[
    (df["predicted"] == "OL") &
    (df["pos > neg"])
].copy()

print(f"OL & pos > neg total: {len(ol_pos_win_df)}")

ol_pos_win_df = ol_pos_win_df.sort_values(
    by="clip_margin",
    ascending=False
)

def check_top_percent(df_sub, percent):
    k = int(len(df_sub) * percent / 100)
    top_df = df_sub.head(k)

    correct = (top_df["true_label"] == top_df["random_label"]).sum()

    return {
        "percent": percent,
        "top_k": k,
        "correct": correct,
        "ratio": correct / k if k > 0 else 0
    }

results = []

for p in [5, 10, 20]:
    res = check_top_percent(ol_pos_win_df, p)
    results.append(res)

for r in results:
    print(
        f"Top {r['percent']}% OL & pos > neg: "
        f"{r['correct']}/{r['top_k']} "
        f"({r['ratio']:.2%}) correct"
    )

summary = pd.DataFrame(results)
summary

OL & pos > neg total: 3968
Top 5% OL & pos > neg: 160/198 (80.81%) correct
Top 10% OL & pos > neg: 288/396 (72.73%) correct
Top 20% OL & pos > neg: 515/793 (64.94%) correct


,percent,top_k,correct,ratio
0,5,198,160,0.808081
1,10,396,288,0.727273
2,20,793,515,0.649433


# Phân tích cho : predicted = CL

In [18]:
cl_df = df[df["predicted"] == "CL"]

cl_total = len(cl_df)

cl_pos_win = cl_df["pos > neg"].sum()
cl_neg_win = cl_df["neg > pos"].sum()
cl_equal   = cl_df["pos == neg"].sum()

print(f"CL total: {cl_total}")
print(f"CL pos > neg: {cl_pos_win} ({cl_pos_win/cl_total:.2%})")
print(f"CL neg > pos: {cl_neg_win} ({cl_neg_win/cl_total:.2%})")
print(f"CL equal:     {cl_equal} ({cl_equal/cl_total:.2%})")


print("\n==================================================\n")


cl_correct = df[
    (df["predicted"] == "CL") &
    (df["true_label"] != df["random_label"])
]

cl_correct_total = len(cl_correct)
cl_correct_neg_win = cl_correct["neg > pos"].sum()

print(f"CL (correct): {cl_correct_neg_win}/{cl_correct_total} "
      f"({cl_correct_neg_win / cl_correct_total:.2%}) neg > pos")



CL total: 43385
CL pos > neg: 11001 (25.36%)
CL neg > pos: 32276 (74.39%)
CL equal:     108 (0.25%)


CL (correct): 31904/42659 (74.79%) neg > pos
